**arcasHLA typing**
---

From [arcasHLA](https://github.com/RabadanLab/arcasHLA/tree/master).

In [4]:
from arcas_hla_runner import ArcasHLARunner

In [5]:
runner = ArcasHLARunner()
runner.set_arcas_hla_path("/home/jovyan/arcasHLA/arcasHLA")

Current reference database version: # file: release_version.txt
# date: 2025-01-15
# version: IPD-IMGT/HLA 3.59.0
# origin: https://github.com/ANHIG/IMGTHLA/release_version.txt
# repository: https://raw.githubusercontent.com/ANHIG/IMGTHLA/Latest/release_version.txt
# author: IPD Team (ipdsubs@anthonynolan.org)


In [24]:
# Step 1: Extract reads
runner.extract_reads("samples/GC130443/GC130443.bam", "samples/GC130443/", threads=8, verbose=True, single=True)


--------------------------------------------------------------------------------
[log] Date: 2025-01-24
[log] Sample: GC130443
[log] Input file: samples/GC130443/GC130443.bam
[log] Read type: single-end
--------------------------------------------------------------------------------
[extract] Extracting reads from samples/GC130443/GC130443.bam
[extract] Extracting chromosome 6: 

	samtools view -H -@8 samples/GC130443/GC130443.bam -o /tmp/arcas_2af4c260-f74c-47e1-b06b-3722b7b64034/GC130443.hla.sam

[extract] Extracting chromosome 6: 

	samtools view -@8 -F 4 samples/GC130443/GC130443.bam 6 >> /tmp/arcas_2af4c260-f74c-47e1-b06b-3722b7b64034/GC130443.hla.sam

[extract] Sorting bam: 

	samtools sort -n -@8 /tmp/arcas_2af4c260-f74c-47e1-b06b-3722b7b64034/GC130443.hla.sam -o /tmp/arcas_2af4c260-f74c-47e1-b06b-3722b7b64034/GC130443.hla.sorted.bam

	[bam_sort_core] merging from 0 files and 8 in-memory blocks...
	
[extract] Converting bam to fastq: 

	bedtools bamtofastq -i /tmp/arcas_2af4c26

In [37]:
# Step 2: Genotype
fq_file = "fastq/GC130443/GC130443.221208_Donor_variability_in_cartilage_microtissues.230203.HiSeq4000.FCA.lane7.gcap_22_10.I1.fastq.gz"
output_dir = "fastq/GC130443/GC130443.221208_Donor_variability_output_genotype"

runner.genotype(fq_file, output_dir,
                genes="A,B,C,DPB1,DQB1,DQA1,DRB1", 
                threads=8, verbose=True, single=True)


--------------------------------------------------------------------------------
[log] Date: 2025-01-24
[log] Sample: GC130443
[log] Input file(s): fastq/GC130443/GC130443.221208_Donor_variability_in_cartilage_microtissues.230203.HiSeq4000.FCA.lane7.gcap_22_10.I1.fastq.gz
[log] Reference: f80df4e0e3bf7dbb3c229e12e0dc2f63dbe7292f
--------------------------------------------------------------------------------
[alignment] Analyzing read length
[alignment] Pseudoaligning with Kallisto: 

	kallisto pseudo -i /home/jovyan/arcasHLA/scripts/../dat/ref/hla.idx -t 8 -o /tmp/arcas_a9c1a4ac-0ed6-4c09-8132-628dcb534b1f/ --single -l 200 -s 20 fastq/GC130443/GC130443.221208_Donor_variability_in_cartilage_microtissues.230203.HiSeq4000.FCA.lane7.gcap_22_10.I1.fastq.gz

	
	[quant] fragment length distribution is truncated gaussian with mean = 200, sd = 20
	[index] k-mer length: 31
	[index] number of targets: 21,476
	[index] number of k-mers: 402,141
	[index] number of equivalence classes: 44,815
	[qua

In [41]:
# Step 3: Partial Genotyping
fq_file = "fastq/GC130443/GC130443.221208_Donor_variability_in_cartilage_microtissues.230203.HiSeq4000.FCA.lane7.gcap_22_10.I1.fastq.gz"
genotype_file = "fastq/GC130443/GC130443.221208_Donor_variability_output_genotype/GC130443.genotype.json"
output_dir = "fastq/GC130443/GC130443.221208_Donor_variability_output_partial"

runner.partial_genotype(fq_file, 
                        "A,B,C,DPB1,DQB1,DQA1,DRB1",
                        genotype_file,
                        output_dir,
                        threads=8, verbose=True, single=True)


--------------------------------------------------------------------------------
[log] Date: 2025-01-24
[log] Sample: GC130443
[log] Input file(s): fastq/GC130443/GC130443.221208_Donor_variability_in_cartilage_microtissues.230203.HiSeq4000.FCA.lane7.gcap_22_10.I1.fastq.gz
[log] Reference: f80df4e0e3bf7dbb3c229e12e0dc2f63dbe7292f
--------------------------------------------------------------------------------
[alignment] Analyzing read length
[alignment] Pseudoaligning with Kallisto: 

	kallisto pseudo -i /home/jovyan/arcasHLA/scripts/../dat/ref/hla_partial.idx -t 8 -o /tmp/arcas_731f5e09-eaa8-49f6-b690-b65efa752a87/ --single -l 200 -s 20 fastq/GC130443/GC130443.221208_Donor_variability_in_cartilage_microtissues.230203.HiSeq4000.FCA.lane7.gcap_22_10.I1.fastq.gz

	
	[quant] fragment length distribution is truncated gaussian with mean = 200, sd = 20
	[index] k-mer length: 31
	[index] number of targets: 177,638
	[index] number of k-mers: 555,347
	[index] number of equivalence classes: 224

In [41]:
# Step 3: Partial Genotyping
fq_file = "fastq/GC130443/GC130443.221208_Donor_variability_in_cartilage_microtissues.230203.HiSeq4000.FCA.lane7.gcap_22_10.I1.fastq.gz"
genotype_file = "fastq/GC130443/GC130443.221208_Donor_variability_output_genotype/GC130443.genotype.json"
output_dir = "fastq/GC130443/GC130443.221208_Donor_variability_output_partial"

runner.partial_genotype(fq_file, 
                        "A,B,C,DPB1,DQB1,DQA1,DRB1",
                        genotype_file,
                        output_dir,
                        threads=8, verbose=True, single=True)


--------------------------------------------------------------------------------
[log] Date: 2025-01-24
[log] Sample: GC130443
[log] Input file(s): fastq/GC130443/GC130443.221208_Donor_variability_in_cartilage_microtissues.230203.HiSeq4000.FCA.lane7.gcap_22_10.I1.fastq.gz
[log] Reference: f80df4e0e3bf7dbb3c229e12e0dc2f63dbe7292f
--------------------------------------------------------------------------------
[alignment] Analyzing read length
[alignment] Pseudoaligning with Kallisto: 

	kallisto pseudo -i /home/jovyan/arcasHLA/scripts/../dat/ref/hla_partial.idx -t 8 -o /tmp/arcas_731f5e09-eaa8-49f6-b690-b65efa752a87/ --single -l 200 -s 20 fastq/GC130443/GC130443.221208_Donor_variability_in_cartilage_microtissues.230203.HiSeq4000.FCA.lane7.gcap_22_10.I1.fastq.gz

	
	[quant] fragment length distribution is truncated gaussian with mean = 200, sd = 20
	[index] k-mer length: 31
	[index] number of targets: 177,638
	[index] number of k-mers: 555,347
	[index] number of equivalence classes: 224